In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/logistics-shopee-code-league/SLA_matrix.xlsx
/kaggle/input/logistics-shopee-code-league/delivery_orders_march.csv


In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
df = pd.read_csv('../input/logistics-shopee-code-league/delivery_orders_march.csv')

In [4]:
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [5]:
df['2nd_deliver_attempt'] = pd.to_datetime(df['2nd_deliver_attempt'], unit='s')
df['1st_deliver_attempt'] = pd.to_datetime(df['1st_deliver_attempt'], unit='s')
df['pick'] = pd.to_datetime(df['pick'], unit='s')
df

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,2020-03-02 08:39:57,2020-03-05 05:09:18,NaT,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,2020-03-04 08:19:28,2020-03-06 02:53:56,2020-03-10 00:02:56,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,2020-03-04 07:20:34,2020-03-06 01:56:19,NaT,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,2020-03-05 14:36:56,2020-03-07 04:45:41,NaT,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,2020-03-04 10:38:25,2020-03-06 07:41:40,NaT,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...
...,...,...,...,...,...,...
3176308,31504087640510,2020-04-02 10:02:08,2020-04-04 05:22:22,NaT,"mayapis. edeliza JC274 5542 brgy.Dita, Metro M...",Intimate St) compound. Sekiat 2B8 98-B Mel far...
3176309,31504147352227,2020-04-03 06:23:31,2020-04-06 13:17:27,2020-04-07 03:52:11,"Superdome PORKY K307 Libertad, PCGCC sub.bagui...","13th 3616 Pilipinas, :1v26 1f-5 (Village) bl10..."
3176310,31504462290482,2020-04-02 11:29:11,2020-04-04 01:55:40,NaT,Galay's 1republic 333-18 Diliman 74d Malbay fe...,"Lane, 379B 1739A Ivc AFPFC Cainta, Waterpark, ..."
3176311,31504851495943,2020-04-03 05:19:54,2020-04-07 03:19:12,NaT,"#4560, 80south biloca vi!!age KM21, MANRESA, ,...",lrt-2 Consolacion 224-A Paradise 6D 36st champ...


In [6]:
def difference(row):
    pickup= row['pick'].date()
    first_attempt = row['1st_deliver_attempt'].date()
    diff = first_attempt-pickup
    return diff.days

In [7]:
df['difference']=df.apply(difference,axis=1)

In [8]:
df[0:10]

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,difference
0,2215676524,2020-03-02 08:39:57,2020-03-05 05:09:18,NaT,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,3
1,2219624609,2020-03-04 08:19:28,2020-03-06 02:53:56,2020-03-10 00:02:56,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",2
2,2220979489,2020-03-04 07:20:34,2020-03-06 01:56:19,NaT,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",2
3,2221066352,2020-03-05 14:36:56,2020-03-07 04:45:41,NaT,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",2
4,2222478803,2020-03-04 10:38:25,2020-03-06 07:41:40,NaT,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,2
5,2222597288,2020-03-04 13:25:20,2020-03-07 07:41:49,NaT,C-846 park&shop opens Valeda/Sunrise) CompuWar...,33 sta.mesa 1929 hawthorn Metro Manila,3
6,2222738456,2020-03-02 11:31:52,2020-03-05 05:22:32,2020-03-09 00:33:41,"Ka-insong stockholme Lhuilier, polanco. malen ...","salcedo Midlands 980 WALL, 16C paraiso Mall（Ph...",3
7,2224695304,2020-03-02 05:44:01,2020-03-10 10:40:07,NaT,Reel zambo.ciyy puyat.cor road/infront #129A L...,Polar 75 mulawinan 1G-3 señora Text/Call 2P-01...,8
8,2224704587,2020-03-04 07:42:59,2020-03-05 06:14:26,2020-03-09 00:03:03,"Lumber,Naprada POT kadir Bagyan blanga FIRENDS...",skies dept. Crystal apello Stockroom.Star (DAG...,1
9,2225138267,2020-03-04 15:43:31,2020-03-10 04:13:06,NaT,"Hengtong Buhaynasapa, Lask Southville8A Colgat...",9x Comfort Greenheights Lifehomes AANI Metro M...,6


In [9]:
def difference_2(row):
    first_attempt= row['1st_deliver_attempt'].date()
    second_attempt = row['2nd_deliver_attempt']
    x = np.datetime64('NaT')
    
    if pd.isnull(second_attempt):
        return 0
    else:
        diff = second_attempt.date()-first_attempt
        return diff.days
#     print(second_attempt)

In [10]:
df['difference_2']=df.apply(difference_2,axis=1)

In [11]:
# df['is_late'] = '0'
# df.loc[df["Time Difference 2"] > '3 days', 'is_late'] = 1

In [12]:
def extract_address_function(row):
    origins = ['metro manila','luzon','visayas','mindanao']
    origin_map = {'metro manila':0, 'luzon':1,'visayas':2,'mindanao':3}
    SLA = {
        0: {0: 3, 1:5, 2: 7,3:7},
        1: {0: 5, 1:5, 2: 7,3:7},
        2: {0: 5, 1:7, 2: 7,3:7},
        3: {0: 7, 1:7, 2: 7,3:7}
    }
    
    buyer_address = row['buyeraddress'].lower()
    seller_address = row['selleraddress'].lower()
    
    b = ''
    s = ''
    
    for origin in origins:
#         print(buyer_address,'\n', seller_address,'origin',origin)
        if re.search(origin,buyer_address):
            b=origin_map[origin]
        if re.search(origin,seller_address):
            s=origin_map[origin]
    
#     print("Buyer: ",b,"Seller: ",s)
#     print(SLA[b][s])
    return SLA[b][s]
    
#     print('Buyer',buyer_address.lower(), 'Seller: ',seller_address.lower())
# extract_address_function()

In [13]:
df['SLA']=df.apply(extract_address_function,axis=1)

In [14]:
def public_holiday(row):
    public_holidays = ['2020-03-08T12','2020-03-15T12','2020-03-22T12','2020-03-25T12','2020-03-29T12','2020-03-30T12','2020-03-31T12']
    pickup = row['pick'].date() #2020-03-02 08:39:57
#     one_business_days = 1 * pd.offsets.BDay()
    first_attempt = row['1st_deliver_attempt'].date()
#     pickup = one_business_days.apply(pickup)
    
    count = 0
    for public in public_holidays:
        pub = pd.Timestamp(public).date()
        if (pickup<=pub<=first_attempt):
            count+=1
    return count
    
#     print(count)

In [15]:
def public_holiday_1(row):
    public_holidays = ['2020-03-08T12','2020-03-15T12','2020-03-22T12','2020-03-25T12','2020-03-29T12','2020-03-30T12','2020-03-31T12']
    first_attempt = row['1st_deliver_attempt'].date() #2020-03-02 08:39:57
#     one_business_days = 1 * pd.offsets.BDay()
    second_attempt = row['2nd_deliver_attempt'].date()
#     pickup = one_business_days.apply(pickup)
    
    count = 0
    for public in public_holidays:
        pub = pd.Timestamp(public).date()
        if (first_attempt<=pub<=second_attempt):
            count+=1
    return count
    
#     print(count)

In [16]:
df['pub_1']=df.apply(public_holiday,axis=1)
df['pub_2']=df.apply(public_holiday_1,axis=1)
df

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,difference,difference_2,SLA,pub_1,pub_2
0,2215676524,2020-03-02 08:39:57,2020-03-05 05:09:18,NaT,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,3,0,3,0,0
1,2219624609,2020-03-04 08:19:28,2020-03-06 02:53:56,2020-03-10 00:02:56,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",2,4,3,0,1
2,2220979489,2020-03-04 07:20:34,2020-03-06 01:56:19,NaT,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",2,0,3,0,0
3,2221066352,2020-03-05 14:36:56,2020-03-07 04:45:41,NaT,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",2,0,3,0,0
4,2222478803,2020-03-04 10:38:25,2020-03-06 07:41:40,NaT,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,2,0,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...
3176308,31504087640510,2020-04-02 10:02:08,2020-04-04 05:22:22,NaT,"mayapis. edeliza JC274 5542 brgy.Dita, Metro M...",Intimate St) compound. Sekiat 2B8 98-B Mel far...,2,0,3,0,0
3176309,31504147352227,2020-04-03 06:23:31,2020-04-06 13:17:27,2020-04-07 03:52:11,"Superdome PORKY K307 Libertad, PCGCC sub.bagui...","13th 3616 Pilipinas, :1v26 1f-5 (Village) bl10...",3,1,3,0,0
3176310,31504462290482,2020-04-02 11:29:11,2020-04-04 01:55:40,NaT,Galay's 1republic 333-18 Diliman 74d Malbay fe...,"Lane, 379B 1739A Ivc AFPFC Cainta, Waterpark, ...",2,0,3,0,0
3176311,31504851495943,2020-04-03 05:19:54,2020-04-07 03:19:12,NaT,"#4560, 80south biloca vi!!age KM21, MANRESA, ,...",lrt-2 Consolacion 224-A Paradise 6D 36st champ...,4,0,5,0,0


In [17]:
import math
def convert_to_days(row):
    diff = row['difference'] - row['pub_1']
    check = row['difference_2'] - row['pub_2']
    SLA = row['SLA']
    
    if math.isnan(check):
#         print("NAN")
        if diff>SLA:
            return 1
        else:
            return 0
    elif int(check)>3:
#         print("Check >3")
        return 1
    else:
#         print("Check <3")
        return 0
    

In [18]:
df['is_late']=df.apply(convert_to_days,axis=1)
df

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,difference,difference_2,SLA,pub_1,pub_2,is_late
0,2215676524,2020-03-02 08:39:57,2020-03-05 05:09:18,NaT,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,3,0,3,0,0,0
1,2219624609,2020-03-04 08:19:28,2020-03-06 02:53:56,2020-03-10 00:02:56,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",2,4,3,0,1,0
2,2220979489,2020-03-04 07:20:34,2020-03-06 01:56:19,NaT,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",2,0,3,0,0,0
3,2221066352,2020-03-05 14:36:56,2020-03-07 04:45:41,NaT,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",2,0,3,0,0,0
4,2222478803,2020-03-04 10:38:25,2020-03-06 07:41:40,NaT,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,2,0,5,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3176308,31504087640510,2020-04-02 10:02:08,2020-04-04 05:22:22,NaT,"mayapis. edeliza JC274 5542 brgy.Dita, Metro M...",Intimate St) compound. Sekiat 2B8 98-B Mel far...,2,0,3,0,0,0
3176309,31504147352227,2020-04-03 06:23:31,2020-04-06 13:17:27,2020-04-07 03:52:11,"Superdome PORKY K307 Libertad, PCGCC sub.bagui...","13th 3616 Pilipinas, :1v26 1f-5 (Village) bl10...",3,1,3,0,0,0
3176310,31504462290482,2020-04-02 11:29:11,2020-04-04 01:55:40,NaT,Galay's 1republic 333-18 Diliman 74d Malbay fe...,"Lane, 379B 1739A Ivc AFPFC Cainta, Waterpark, ...",2,0,3,0,0,0
3176311,31504851495943,2020-04-03 05:19:54,2020-04-07 03:19:12,NaT,"#4560, 80south biloca vi!!age KM21, MANRESA, ,...",lrt-2 Consolacion 224-A Paradise 6D 36st champ...,4,0,5,0,0,0


In [19]:
answer = df[['orderid','is_late']]
answer.to_csv('solution_4.csv',index=False)

In [20]:
answer[400:500]

,orderid,is_late
400,2229908542,0
401,2229922762,0
402,2229925168,0
403,2229928458,0
404,2229940888,0
...,...,...
495,2230553867,0
496,2230554874,0
497,2230560411,0
498,2230574895,0
